In [ ]:
!nvidia-smi

32
Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          39 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   32
  On-line CPU(s) list:    0-31
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Core(TM) i9-14900HX
    CPU family:           6
    Model:                183
    Thread(s) per core:   2
    Core(s) per socket:   24
    Socket(s):            1
    Stepping:             1
    CPU(s) scaling MHz:   25%
    CPU max MHz:          5800.0000
    CPU min MHz:          800.0000
    BogoMIPS:             4838.40
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 s
                          s ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc 
                          art arch_perfmon pebs bts rep_good nopl xtopology nons
                          top_tsc cpuid

In [19]:
# Enable all CPU cores
from tqdm import tqdm
import pandas as pd
import re
import pandas as pd

# Read the file content
def parse_to_datafame(file_path):
  with open(file_path, "r") as file:
      content = file.read()
  # Define a regex pattern to extract the test case and its details
  pattern = r"\s+structure\s+(?P<structure>.+)\s+nat\s+(?P<nat>\d+)\s+nprim\s+(?P<nprim>\d+)\s+maxl\s+(?P<maxl>\d+)\s+max_nnl\s+(?P<max_nnl>\d+)\s+mean_nnl\s+(?P<mean_nnl>\d+)\s+max_nsh\s+(?P<max_nsh>\d+)\s+cpu_time\s+(?P<cpu_time>[\d.]+)\s+gpu_gb_in\s+(?P<gpu_gb_in>[\d.]+)\s+gpu_gb_out\s+(?P<gpu_gb_out>[\d.]+)\s+gpu_gb_total\s+(?P<gpu_gb_total>[\d.]+)\s+gpu_between_atoms\s+(?P<gpu_between_atoms>[\d.]+)\s+gpu_in_atoms\s+(?P<gpu_in_atoms>[\d.]+)\s+gpu_time\s+(?P<gpu_time>[\d.]+)\s+gpu_transfer_time\s+(?P<gpu_transfer_time>[\d.]+)"

  # Use re.finditer to extract all matches
  matches = re.finditer(pattern, content)

  # Create a list of dictionaries to store the extracted data
  data = []
  for match in matches:
      data.append(match.groupdict())

  # Convert the list of dictionaries into a pandas DataFrame
  df = pd.DataFrame(data)

# Convert numeric columns to appropriate data types
  numeric_columns = [
    'nat', 'nprim', 'maxl', 'max_nnl', 'mean_nnl', 'max_nsh',
    'cpu_time', 'gpu_gb_in', 'gpu_gb_out', 'gpu_gb_total',
    'gpu_between_atoms', 'gpu_in_atoms', 'gpu_time', 'gpu_transfer_time'
  ]
  # # Convert the numeric columns to float
  df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)

  # Display the DataFrame
  df = df.sort_values('nat')
  return df

option = 'RTX4060'
!mkdir -p notes/capture/{option}
!mkdir -p notes/data/{option}
!meson test -C build/ hamiltonian --verbose -t 0 > notes/capture/{option}/gpu.txt
df = parse_to_datafame(f"notes/capture/{option}/gpu.txt")
df.to_csv(f"notes/data/{option}/gpu.csv", index=False)
df

,structure,nat,nprim,maxl,max_nnl,mean_nnl,max_nsh,cpu_time,gpu_gb_in,gpu_gb_out,gpu_gb_total,gpu_between_atoms,gpu_in_atoms,gpu_time,gpu_transfer_time
0,GLU,29,6,1,28,14,2,11.985369,0.000453,0.000444,0.000896,10.739712,2.131936,12.871648,0.954560
5,alkane_64,192,6,1,30,20,2,8.414220,0.013026,0.012976,0.026002,9.414656,1.933568,11.348224,6.645632
6,alkane_128,384,6,1,30,21,2,14.909392,0.052005,0.051905,0.103909,17.855583,2.703648,20.559231,21.178593
1,dna,387,6,2,165,69,3,59.047182,0.232068,0.231803,0.463871,212.961273,9.908224,222.869492,87.494461
3,101d-netropsin-and-dna,556,6,2,484,203,3,105.978783,0.479133,0.478153,0.957286,383.203156,10.908800,394.111969,166.830170
7,alkane_256,768,6,1,30,21,2,32.171431,0.207819,0.207618,0.415437,50.085888,5.079040,55.164928,90.081284
2,1lyz-lysozyme,1102,6,2,311,99,3,210.940091,1.749900,1.748891,3.498791,541.932922,15.230976,557.163879,575.938538
4,103l-hydrolase,1404,6,2,309,99,3,320.774930,2.816439,2.815150,5.631588,673.622864,17.464289,691.087158,962.728943
8,alkane_512,1536,6,1,30,21,2,86.739008,0.830875,0.830472,1.661347,71.779327,6.709184,78.488510,292.157104
9,alkane_1024,3072,6,1,30,21,2,290.792890,3.322695,3.321889,6.644584,137.043198,11.519104,148.562302,1141.643188
